## Problem Statement

We have been given a ODE: 

$$
\frac{d^2x(t)}{dt^2} = - k x(t)
$$

and boundary condition: 

$$
x(0) = 5
$$


- Independent variables: t (input)
- Dependent variables: x (outputs)


We have to find out u(x,t) for all t in range [0,10]

PINNs:

$$
\frac{d^2N(t)}{dt^2} = - k N(t)
$$



In [ ]:
import numpy as np
import plotly.graph_objects as go

import torch
import torch.nn as nn
from torch.autograd import Variable

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def build_model(input_dimension, hidden_dimension, output_dimension):
    """Build a neural network of given dimensions."""

    modules=[]
    modules.append(torch.nn.Linear(input_dimension, hidden_dimension[0]))
    modules.append(torch.nn.Tanh())
    for i in range(len(hidden_dimension)-1):
        modules.append(torch.nn.Linear(hidden_dimension[i], hidden_dimension[i+1]))
        modules.append(torch.nn.Tanh())
    
    modules.append(torch.nn.Linear(hidden_dimension[-1], output_dimension))
    
    model = torch.nn.Sequential(*modules)

    return model

In [ ]:
net = build_model(1,[40, 40, 20, 20],1)
net = net.to(device)
mse_cost_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters())

In [ ]:
def get_derivative(x, t, n=1):
    
    if n == 0:
        return x
    else:
        
        dxdt = torch.autograd.grad(x, t, torch.ones(t.size()[0], 1, device=device),
        create_graph=True,
        retain_graph=True, 
        #allow_unused=True,
        )[0]
    return get_derivative(dxdt, t, n - 1)

In [ ]:
def f(t, net, k):
    u = net(t)
    dudtdt = get_derivative(u, t, 2)
    ode = dudtdt + k*u
    return ode

In [ ]:
k = torch.tensor([[3]], dtype=torch.float32, device=device)
init_t = torch.tensor([[0]], dtype=torch.float32, device=device)
init_x = torch.tensor([[5]], dtype=torch.float32, device=device)

n_points = 5000
t_collocation = np.linspace(0, 20, n_points).reshape(-1,1)
all_zeros = np.zeros((n_points,1))

pt_t_collocation = Variable(torch.from_numpy(t_collocation).float(), requires_grad=True).to(device)
pt_all_zeros = Variable(torch.from_numpy(all_zeros).float(), requires_grad=True).to(device)

In [ ]:

iterations = 100000
loss_count = []


for epoch in range(iterations):
    optimizer.zero_grad() 

    init_pred = net(init_t)
    mse_b = mse_cost_function(init_pred, init_x)
    
    f_out = f(pt_t_collocation, net,k)
    mse_f = mse_cost_function(f_out, pt_all_zeros)

    loss = mse_b + mse_f
    loss_count.append(loss.item())

    loss.backward(retain_graph=True)
    optimizer.step()

    with torch.autograd.no_grad():
        if epoch % 1000 ==0:
    	    print(epoch,"Traning Loss:",loss.data)

0 Traning Loss: tensor(24.5573, device='cuda:0')
1000 Traning Loss: tensor(1.6296, device='cuda:0')
2000 Traning Loss: tensor(1.4341, device='cuda:0')
3000 Traning Loss: tensor(1.3464, device='cuda:0')
4000 Traning Loss: tensor(1.3008, device='cuda:0')
5000 Traning Loss: tensor(1.2557, device='cuda:0')
6000 Traning Loss: tensor(1.1995, device='cuda:0')
7000 Traning Loss: tensor(1.0418, device='cuda:0')
8000 Traning Loss: tensor(0.9241, device='cuda:0')
9000 Traning Loss: tensor(0.8960, device='cuda:0')
10000 Traning Loss: tensor(0.8515, device='cuda:0')
11000 Traning Loss: tensor(0.8350, device='cuda:0')
12000 Traning Loss: tensor(0.8169, device='cuda:0')
13000 Traning Loss: tensor(0.7751, device='cuda:0')
14000 Traning Loss: tensor(0.7724, device='cuda:0')
15000 Traning Loss: tensor(0.6457, device='cuda:0')
16000 Traning Loss: tensor(0.6215, device='cuda:0')
17000 Traning Loss: tensor(0.6265, device='cuda:0')
18000 Traning Loss: tensor(0.6139, device='cuda:0')
19000 Traning Loss: tens

In [ ]:
f_out = net(pt_t_collocation)

In [ ]:
xx = f_out[:, 0].data.cpu().numpy()

In [ ]:
fig = go.Figure()
fig.add_scatter(x = t_collocation.flatten(), y=xx)

In [ ]:
fig = go.Figure()
fig.add_scatter(y = loss_count)

In [ ]:
test_t = np.array([[0]], dtype=np.float32)
t = Variable(torch.from_numpy(test_t), requires_grad=True).to(device)

net(t)

tensor([[4.9976]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
result  = []
for i in range(0, 100):
  test_t = np.array([[i]], dtype=np.float32)
  t = Variable(torch.from_numpy(test_t), requires_grad=True).to(device)
  result.append(net(t).item())

In [ ]:

t_collocation = np.linspace(0, 100, n_points).reshape(-1,1)
pt_t_collocation = Variable(torch.from_numpy(t_collocation).float(), requires_grad=True).to(device)
f_out = net(pt_t_collocation)

xx = f_out[:, 0].data.cpu().numpy()

#Prediction test:

In [ ]:
fig = go.Figure()

fig.add_scatter(x = t_collocation.flatten(), y=xx)